# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_id_text = []
    for sentence in source_text.split('\n'):
        source_sentence = []
        for word in sentence.split():
            if word in source_vocab_to_int:
                source_sentence.append(source_vocab_to_int[word])
        source_id_text.append(source_sentence)
        
    target_id_text = []
    for sentence in target_text.split('\n'):
        target_sentence = []
        for word in sentence.split():
            if word in target_vocab_to_int:
                target_sentence.append(target_vocab_to_int[word])
        target_sentence.append(target_vocab_to_int['<EOS>'])
        target_id_text.append(target_sentence)
    
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [40]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.2.1
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32, shape=(None, None), name='input')
    targets = tf.placeholder(tf.int32, shape=(None, None))
    learning_rate = tf.placeholder(tf.float32, shape=())
    keep_probability = tf.placeholder(tf.float32, shape=(), name='keep_prob')
    target_sequence_length = tf.placeholder(tf.int32, shape=(None,), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, shape=(None,), name='source_sequence_length')
    return input, targets, learning_rate, keep_probability, target_sequence_length, max_target_sequence_length, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, 
                                                       source_vocab_size, 
                                                       encoding_embedding_size, 
                                                       initializer=tf.random_uniform_initializer(-1, 1))
    
    def make_lstm_cell(rnn_size):
        enc_lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        return tf.contrib.rnn.DropoutWrapper(enc_lstm)
    
    enc_cells = tf.contrib.rnn.MultiRNNCell([make_lstm_cell(rnn_size) for _ in range(num_layers)])
    enc_outputs, enc_state = tf.nn.dynamic_rnn(enc_cells, 
                                               enc_embed_input, 
                                               sequence_length=source_sequence_length, 
                                               dtype=tf.float32)
    
    return enc_outputs, enc_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    training_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, 
                                                        target_sequence_length, 
                                                        time_major=False)
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                                       training_helper, 
                                                       encoder_state, 
                                                       output_layer)

    return tf.contrib.seq2seq.dynamic_decode(training_decoder, 
                                             impute_finished=True, 
                                             maximum_iterations=max_summary_length)[0]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [36]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function\
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')

    infer_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, 
                                                            start_tokens, 
                                                            end_of_sequence_id)
    infer_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                                    infer_helper, 
                                                    encoder_state, 
                                                    output_layer=output_layer)

    return tf.contrib.seq2seq.dynamic_decode(infer_decoder, 
                                             impute_finished=True, 
                                             maximum_iterations=max_target_sequence_length)[0]



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

momo
Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [41]:
from tensorflow.python.layers.core import Dense

def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    dec_embeddings = tf.Variable(
        tf.random_uniform([target_vocab_size, decoding_embedding_size], minval=-1, maxval=1))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    def make_lstm_cell(rnn_size):
        lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size)
        return tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
    dec_cells = tf.contrib.rnn.MultiRNNCell([make_lstm_cell(rnn_size) for _ in range(num_layers)])
    
    output_layer = Dense(target_vocab_size, 
                         kernel_initializer=tf.truncated_normal_initializer(stddev=0.1))
    
    training_decoder_logits = None
    infer_decoder_logits = None
    
    with tf.variable_scope('decode'):
        training_decoder_logits = decoding_layer_train(encoder_state, 
                                                       dec_cells, 
                                                       dec_embed_input, 
                                                       target_sequence_length,
                                                       max_target_sequence_length, 
                                                       output_layer, 
                                                       keep_prob)
    with tf.variable_scope('decode', reuse=True):
        infer_decoder_logits = decoding_layer_infer(encoder_state, 
                                                    dec_cells, 
                                                    dec_embeddings, 
                                                    target_vocab_to_int['<GO>'], 
                                                    target_vocab_to_int['<EOS>'], 
                                                    max_target_sequence_length, 
                                                    target_vocab_size, 
                                                    output_layer, 
                                                    batch_size, 
                                                    keep_prob)
    return training_decoder_logits, infer_decoder_logits



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

momo
Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [42]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    _, enc_state = encoding_layer(input_data, 
                                  rnn_size, 
                                  num_layers, 
                                  keep_prob, 
                                  source_sequence_length, 
                                  source_vocab_size, 
                                  enc_embedding_size)
    
    dec_input = process_decoder_input(target_data, 
                                      target_vocab_to_int, 
                                      batch_size)
    
    training_decoder_output, infer_decoder_output = decoding_layer(dec_input, 
                                                                   enc_state, 
                                                                   target_sequence_length, 
                                                                   max_target_sentence_length, 
                                                                   rnn_size, 
                                                                   num_layers, 
                                                                   target_vocab_to_int, 
                                                                   target_vocab_size, 
                                                                   batch_size, 
                                                                   keep_prob, 
                                                                   dec_embedding_size)
    
    return training_decoder_output, infer_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

momo
Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [43]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 500
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 200
decoding_embedding_size = 200
# Learning Rate
learning_rate = 0.01
# Dropout Keep Probability
keep_probability = 0.5
display_step = 7

### Build the Graph
Build the graph using the neural network you implemented.

In [44]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


momo


Batch and pad the source and target sequences

In [45]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [46]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    7/1077 - Train Accuracy: 0.2258, Validation Accuracy: 0.3050, Loss: 4.4622
Epoch   0 Batch   14/1077 - Train Accuracy: 0.2734, Validation Accuracy: 0.3232, Loss: 3.6801
Epoch   0 Batch   21/1077 - Train Accuracy: 0.2820, Validation Accuracy: 0.3498, Loss: 3.4575
Epoch   0 Batch   28/1077 - Train Accuracy: 0.2188, Validation Accuracy: 0.2923, Loss: 3.0891
Epoch   0 Batch   35/1077 - Train Accuracy: 0.2441, Validation Accuracy: 0.3058, Loss: 2.8794
Epoch   0 Batch   42/1077 - Train Accuracy: 0.0797, Validation Accuracy: 0.0678, Loss: 2.6604
Epoch   0 Batch   49/1077 - Train Accuracy: 0.1518, Validation Accuracy: 0.1481, Loss: 2.4714
Epoch   0 Batch   56/1077 - Train Accuracy: 0.2922, Validation Accuracy: 0.3523, Loss: 2.3427
Epoch   0 Batch   63/1077 - Train Accuracy: 0.3170, Validation Accuracy: 0.3430, Loss: 2.0564
Epoch   0 Batch   70/1077 - Train Accuracy: 0.2081, Validation Accuracy: 0.3072, Loss: 2.1042
Epoch   0 Batch   77/1077 - Train Accuracy: 0.2168, Validati

Epoch   0 Batch  623/1077 - Train Accuracy: 0.5156, Validation Accuracy: 0.5721, Loss: 0.7130
Epoch   0 Batch  630/1077 - Train Accuracy: 0.5773, Validation Accuracy: 0.5891, Loss: 0.7094
Epoch   0 Batch  637/1077 - Train Accuracy: 0.5445, Validation Accuracy: 0.5813, Loss: 0.7225
Epoch   0 Batch  644/1077 - Train Accuracy: 0.5648, Validation Accuracy: 0.5817, Loss: 0.6889
Epoch   0 Batch  651/1077 - Train Accuracy: 0.5647, Validation Accuracy: 0.5891, Loss: 0.6915
Epoch   0 Batch  658/1077 - Train Accuracy: 0.5565, Validation Accuracy: 0.5671, Loss: 0.6597
Epoch   0 Batch  665/1077 - Train Accuracy: 0.5258, Validation Accuracy: 0.5749, Loss: 0.7037
Epoch   0 Batch  672/1077 - Train Accuracy: 0.5766, Validation Accuracy: 0.5945, Loss: 0.6832
Epoch   0 Batch  679/1077 - Train Accuracy: 0.5563, Validation Accuracy: 0.5692, Loss: 0.7068
Epoch   0 Batch  686/1077 - Train Accuracy: 0.5603, Validation Accuracy: 0.5675, Loss: 0.6590
Epoch   0 Batch  693/1077 - Train Accuracy: 0.5177, Validati

Epoch   1 Batch  168/1077 - Train Accuracy: 0.6345, Validation Accuracy: 0.6271, Loss: 0.5662
Epoch   1 Batch  175/1077 - Train Accuracy: 0.6637, Validation Accuracy: 0.6289, Loss: 0.5307
Epoch   1 Batch  182/1077 - Train Accuracy: 0.6484, Validation Accuracy: 0.6353, Loss: 0.5361
Epoch   1 Batch  189/1077 - Train Accuracy: 0.6320, Validation Accuracy: 0.6353, Loss: 0.5367
Epoch   1 Batch  196/1077 - Train Accuracy: 0.6680, Validation Accuracy: 0.6349, Loss: 0.5103
Epoch   1 Batch  203/1077 - Train Accuracy: 0.6133, Validation Accuracy: 0.6378, Loss: 0.5349
Epoch   1 Batch  210/1077 - Train Accuracy: 0.6637, Validation Accuracy: 0.6367, Loss: 0.5279
Epoch   1 Batch  217/1077 - Train Accuracy: 0.6531, Validation Accuracy: 0.6523, Loss: 0.5134
Epoch   1 Batch  224/1077 - Train Accuracy: 0.6496, Validation Accuracy: 0.6470, Loss: 0.5163
Epoch   1 Batch  231/1077 - Train Accuracy: 0.6344, Validation Accuracy: 0.6392, Loss: 0.5398
Epoch   1 Batch  238/1077 - Train Accuracy: 0.6781, Validati

Epoch   1 Batch  784/1077 - Train Accuracy: 0.6977, Validation Accuracy: 0.6957, Loss: 0.4024
Epoch   1 Batch  791/1077 - Train Accuracy: 0.6914, Validation Accuracy: 0.6729, Loss: 0.4234
Epoch   1 Batch  798/1077 - Train Accuracy: 0.6762, Validation Accuracy: 0.6896, Loss: 0.4153
Epoch   1 Batch  805/1077 - Train Accuracy: 0.7039, Validation Accuracy: 0.6783, Loss: 0.4144
Epoch   1 Batch  812/1077 - Train Accuracy: 0.6492, Validation Accuracy: 0.6808, Loss: 0.4176
Epoch   1 Batch  819/1077 - Train Accuracy: 0.7035, Validation Accuracy: 0.6871, Loss: 0.4084
Epoch   1 Batch  826/1077 - Train Accuracy: 0.7072, Validation Accuracy: 0.6836, Loss: 0.3883
Epoch   1 Batch  833/1077 - Train Accuracy: 0.6992, Validation Accuracy: 0.6935, Loss: 0.3982
Epoch   1 Batch  840/1077 - Train Accuracy: 0.6766, Validation Accuracy: 0.6861, Loss: 0.3987
Epoch   1 Batch  847/1077 - Train Accuracy: 0.7203, Validation Accuracy: 0.6861, Loss: 0.4034
Epoch   1 Batch  854/1077 - Train Accuracy: 0.7109, Validati

Epoch   2 Batch  329/1077 - Train Accuracy: 0.7020, Validation Accuracy: 0.7060, Loss: 0.3319
Epoch   2 Batch  336/1077 - Train Accuracy: 0.7035, Validation Accuracy: 0.6850, Loss: 0.3479
Epoch   2 Batch  343/1077 - Train Accuracy: 0.7176, Validation Accuracy: 0.6964, Loss: 0.3404
Epoch   2 Batch  350/1077 - Train Accuracy: 0.6660, Validation Accuracy: 0.6808, Loss: 0.3306
Epoch   2 Batch  357/1077 - Train Accuracy: 0.7411, Validation Accuracy: 0.7230, Loss: 0.3009
Epoch   2 Batch  364/1077 - Train Accuracy: 0.7266, Validation Accuracy: 0.7081, Loss: 0.3482
Epoch   2 Batch  371/1077 - Train Accuracy: 0.6996, Validation Accuracy: 0.6822, Loss: 0.3127
Epoch   2 Batch  378/1077 - Train Accuracy: 0.7184, Validation Accuracy: 0.7156, Loss: 0.3010
Epoch   2 Batch  385/1077 - Train Accuracy: 0.7320, Validation Accuracy: 0.7241, Loss: 0.3232
Epoch   2 Batch  392/1077 - Train Accuracy: 0.7539, Validation Accuracy: 0.7362, Loss: 0.3243
Epoch   2 Batch  399/1077 - Train Accuracy: 0.7237, Validati

...

Epoch   7 Batch  518/1077 - Train Accuracy: 0.8375, Validation Accuracy: 0.8217, Loss: 0.1665
Epoch   7 Batch  525/1077 - Train Accuracy: 0.8434, Validation Accuracy: 0.8196, Loss: 0.1734
Epoch   7 Batch  532/1077 - Train Accuracy: 0.8234, Validation Accuracy: 0.8249, Loss: 0.1846
Epoch   7 Batch  539/1077 - Train Accuracy: 0.8191, Validation Accuracy: 0.8157, Loss: 0.2006
Epoch   7 Batch  546/1077 - Train Accuracy: 0.8391, Validation Accuracy: 0.8111, Loss: 0.2139
Epoch   7 Batch  553/1077 - Train Accuracy: 0.8285, Validation Accuracy: 0.8004, Loss: 0.1920
Epoch   7 Batch  560/1077 - Train Accuracy: 0.8152, Validation Accuracy: 0.8040, Loss: 0.1530
Epoch   7 Batch  567/1077 - Train Accuracy: 0.8535, Validation Accuracy: 0.8068, Loss: 0.1892
Epoch   7 Batch  574/1077 - Train Accuracy: 0.8405, Validation Accuracy: 0.8139, Loss: 0.1871
Epoch   7 Batch  581/1077 - Train Accuracy: 0.8512, Validation Accuracy: 0.8153, Loss: 0.1590
Epoch   7 Batch  588/1077 - Train Accuracy: 0.8496, Validati

Epoch   8 Batch   63/1077 - Train Accuracy: 0.8769, Validation Accuracy: 0.8363, Loss: 0.1526
Epoch   8 Batch   70/1077 - Train Accuracy: 0.8540, Validation Accuracy: 0.8271, Loss: 0.1742
Epoch   8 Batch   77/1077 - Train Accuracy: 0.8547, Validation Accuracy: 0.8221, Loss: 0.1679
Epoch   8 Batch   84/1077 - Train Accuracy: 0.8695, Validation Accuracy: 0.8335, Loss: 0.1561
Epoch   8 Batch   91/1077 - Train Accuracy: 0.8646, Validation Accuracy: 0.8274, Loss: 0.1609
Epoch   8 Batch   98/1077 - Train Accuracy: 0.8590, Validation Accuracy: 0.8374, Loss: 0.1899
Epoch   8 Batch  105/1077 - Train Accuracy: 0.8070, Validation Accuracy: 0.8082, Loss: 0.1884
Epoch   8 Batch  112/1077 - Train Accuracy: 0.8125, Validation Accuracy: 0.8271, Loss: 0.1787
Epoch   8 Batch  119/1077 - Train Accuracy: 0.8336, Validation Accuracy: 0.8263, Loss: 0.1934
Epoch   8 Batch  126/1077 - Train Accuracy: 0.8330, Validation Accuracy: 0.8107, Loss: 0.1619
Epoch   8 Batch  133/1077 - Train Accuracy: 0.8285, Validati

Epoch   8 Batch  679/1077 - Train Accuracy: 0.8302, Validation Accuracy: 0.8239, Loss: 0.1721
Epoch   8 Batch  686/1077 - Train Accuracy: 0.8490, Validation Accuracy: 0.8299, Loss: 0.1624
Epoch   8 Batch  693/1077 - Train Accuracy: 0.7829, Validation Accuracy: 0.8285, Loss: 0.1990
Epoch   8 Batch  700/1077 - Train Accuracy: 0.8383, Validation Accuracy: 0.8310, Loss: 0.1594
Epoch   8 Batch  707/1077 - Train Accuracy: 0.8438, Validation Accuracy: 0.8235, Loss: 0.1798
Epoch   8 Batch  714/1077 - Train Accuracy: 0.8382, Validation Accuracy: 0.8150, Loss: 0.1572
Epoch   8 Batch  721/1077 - Train Accuracy: 0.8254, Validation Accuracy: 0.8285, Loss: 0.1951
Epoch   8 Batch  728/1077 - Train Accuracy: 0.8207, Validation Accuracy: 0.8129, Loss: 0.1774
Epoch   8 Batch  735/1077 - Train Accuracy: 0.8402, Validation Accuracy: 0.8054, Loss: 0.1571
Epoch   8 Batch  742/1077 - Train Accuracy: 0.8641, Validation Accuracy: 0.8164, Loss: 0.1713
Epoch   8 Batch  749/1077 - Train Accuracy: 0.8238, Validati

Epoch   9 Batch  224/1077 - Train Accuracy: 0.8523, Validation Accuracy: 0.8288, Loss: 0.1731
Epoch   9 Batch  231/1077 - Train Accuracy: 0.8762, Validation Accuracy: 0.8129, Loss: 0.1795
Epoch   9 Batch  238/1077 - Train Accuracy: 0.8461, Validation Accuracy: 0.8374, Loss: 0.1553
Epoch   9 Batch  245/1077 - Train Accuracy: 0.8553, Validation Accuracy: 0.8427, Loss: 0.1417
Epoch   9 Batch  252/1077 - Train Accuracy: 0.8957, Validation Accuracy: 0.8416, Loss: 0.1422
Epoch   9 Batch  259/1077 - Train Accuracy: 0.8469, Validation Accuracy: 0.8391, Loss: 0.1770
Epoch   9 Batch  266/1077 - Train Accuracy: 0.8895, Validation Accuracy: 0.8317, Loss: 0.1485
Epoch   9 Batch  273/1077 - Train Accuracy: 0.8393, Validation Accuracy: 0.8242, Loss: 0.1394
Epoch   9 Batch  280/1077 - Train Accuracy: 0.8461, Validation Accuracy: 0.8356, Loss: 0.1597
Epoch   9 Batch  287/1077 - Train Accuracy: 0.8504, Validation Accuracy: 0.8267, Loss: 0.1561
Epoch   9 Batch  294/1077 - Train Accuracy: 0.8619, Validati

Epoch   9 Batch  840/1077 - Train Accuracy: 0.8359, Validation Accuracy: 0.8359, Loss: 0.1605
Epoch   9 Batch  847/1077 - Train Accuracy: 0.8645, Validation Accuracy: 0.8445, Loss: 0.1852
Epoch   9 Batch  854/1077 - Train Accuracy: 0.8250, Validation Accuracy: 0.8423, Loss: 0.1798
Epoch   9 Batch  861/1077 - Train Accuracy: 0.8203, Validation Accuracy: 0.8409, Loss: 0.1535
Epoch   9 Batch  868/1077 - Train Accuracy: 0.8562, Validation Accuracy: 0.8306, Loss: 0.1696
Epoch   9 Batch  875/1077 - Train Accuracy: 0.8586, Validation Accuracy: 0.8285, Loss: 0.1632
Epoch   9 Batch  882/1077 - Train Accuracy: 0.8457, Validation Accuracy: 0.8292, Loss: 0.1659
Epoch   9 Batch  889/1077 - Train Accuracy: 0.8637, Validation Accuracy: 0.8441, Loss: 0.1603
Epoch   9 Batch  896/1077 - Train Accuracy: 0.8454, Validation Accuracy: 0.8398, Loss: 0.1623
Epoch   9 Batch  903/1077 - Train Accuracy: 0.8473, Validation Accuracy: 0.8342, Loss: 0.1624
Epoch   9 Batch  910/1077 - Train Accuracy: 0.8423, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [47]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [48]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [52]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    seq_ = []
    for word in sentence.split():
        if word in vocab_to_int:
            seq_.append(vocab_to_int[word])
        else:
            seq_.append(vocab_to_int['<UNK>'])
    return seq_


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [53]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [42, 175, 8, 202, 47, 195, 153]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [150, 89, 149, 117, 214, 350, 276, 157, 1]
  French Words: il a vu une nouvelle voiture noire . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.